In [ ]:
!pip install -q "tensorflow==2.11.*"
!pip install -q "tensorflow_io==0.28.*"

In [1]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob
import wave
import csv
import tensorflow as tf
import tensorflow_hub as hub
import librosa
from scipy.io import wavfile
from sklearn.model_selection import train_test_split
import tensorflow_io as tfio
import scipy

C:\ProgramData\Anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [ ]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [ ]:
@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav=tf.squeeze(wav,axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
class_names =list(pd.read_csv(class_map_path)['display_name'])

In [2]:
_ = tf.keras.utils.get_file('car.zip',
                        'https://github.com/Boden448/Sound/archive/main.zip',

                        cache_dir='./',
                        cache_subdir='masse',
                        extract=True)

In [ ]:
metadata_file_path='./masse/Sound-main/meta/DOA.csv'
data_dir='./masse/Sound-main/DOA_audio/'

In [ ]:
metadata_df = pd.read_csv(metadata_file_path)
metadata_df.head()

In [ ]:
#WARNING: After running once, dont run again, as the csv file become unsuable and all the files will have to be redownloaded 

updated_metadata = []


with open(metadata_file_path, "r") as file:

    # create a CSV reader object
    reader = csv.DictReader(file)

    # loop through each row in the metadata file
    for row in reader:

        # get the filename from the current row
        filename = row["filename"]

        # construct the path to the corresponding WAV file
        wav_path = os.path.join(data_dir, filename )

        # update the filename column with the path to the WAV file
        row["filename"] = wav_path

        # add the updated row to the list of updated metadata
        updated_metadata.append(row)
        
with open(metadata_file_path, "w") as file:

    # create a CSV writer object
    writer = csv.DictWriter(file, fieldnames=reader.fieldnames)

    # write the updated metadata to the metadata file
    writer.writeheader()
    writer.writerows(updated_metadata)
    
./mass/Sound-main/meta/

In [3]:
csv_dir = "./masse/Sound-main/meta/"
audio_dir= './masse/Sound-main/DOA_audio/'


In [4]:
csv_file = os.path.join(csv_dir, 'DOA.csv')
df = pd.read_csv(csv_file)

In [5]:
audio_files = [os.path.join(audio_dir, f) for f in os.listdir(audio_dir) if f.endswith('.wav')]
audio_data = []
for f in audio_files:
    audio, sr = librosa.load(f, sr=44100, mono=True)
    audio_data.append(audio)


In [6]:
max_length = 44100
padded_audio_data = []
for audio in audio_data:
    if len(audio) >= max_length:
        padded_audio_data.append(audio[:max_length])
    else:
        padded_audio_data.append(np.pad(audio, (0, max_length - len(audio)), mode='constant'))



In [17]:
padded_audio_data = np.array(padded_audio_data)
csv_data = df[['x', 'z', 'theta']].values
pos_data = df[['x', 'z']].values
theta_data = df['theta'].values


print(f"Loaded {len(audio_files)} audio files.")
print(f"Shape of padded audio data: {padded_audio_data.shape}")
print(f"Shape of csv data: {csv_data.shape}")
print(f"Shape of pos data: {pos_data.shape}")
print(f"Shape of theta data: {theta_data.shape}")

Loaded 24 audio files.
Shape of padded audio data: (24, 44100)
Shape of csv data: (24, 3)
Shape of pos data: (24, 2)
Shape of theta data: (24,)


In [14]:
input_audio = tf.keras.layers.Input(shape=(max_length,1))
lstm1 = tf.keras.layers.LSTM(128)(input_audio)
dense1 = tf.keras.layers.Dense(64, activation='softmax')(lstm1)
input_csv = tf.keras.layers.Input(shape=(3,))
concat = tf.keras.layers.Concatenate()([dense1, input_csv])
dense2 = tf.keras.layers.Dense(32, activation='softmax')(concat)
output_pos = tf.keras.layers.Dense(2, activation='linear')(dense2)
output_theta = tf.keras.layers.Dense(1, activation='linear')(dense2)

In [15]:
model = tf.keras.models.Model(inputs=[input_audio, input_csv], outputs=[output_pos, output_theta])
model.compile(loss='mse', optimizer='adam')

In [16]:
model.fit(x=[padded_audio_data.reshape(-1, max_length, 1), csv_data],
          y=[pos_data, theta_data],
          epochs=1,
          batch_size=32,
          validation_split=0.2)

1/1 [==============================] - 65s 65s/step - loss: 129441.6016 - dense_10_loss: 124922.5547 - dense_11_loss: 4519.0576 - val_loss: 150809.7656 - val_dense_10_loss: 124950.4844 - val_dense_11_loss: 25859.2871


In [18]:
def test_model():
    X_audio, X_csv, y_pos, y_theta = load_data()
    model = build_model(max_length=44100)
    model.fit(x=[X_audio.reshape(-1, 44100, 1), X_csv],
              y=[y_pos, y_theta],
              epochs=1,
              batch_size=32,
              validation_split=0.2)

test_model()

NameError: name 'load_data' is not defined

In [ ]:
predicted_pos, predicted_theta = model.predict([test_audio, test_csv])

print('Predicted position: {}'.format(predicted_pos[0]))
print('Predicted angle: {}'.format(predicted_theta[0]))
